In [23]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable 
import pandas as pd
from scipy import signal
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import random
#import seaborn as sns
"""
    number of trajectories in each city
    # austin --  train: 43041 test: 6325 
    # miami -- train: 55029 test:7971
    # pittsburgh -- train: 43544 test: 6361
    # dearborn -- train: 24465 test: 3671
    # washington-dc -- train: 25744 test: 3829
    # palo-alto -- train:  11993 test:1686

    trajectories sampled at 10HZ rate, input 5 seconds, output 6 seconds
    
"""

'\n    number of trajectories in each city\n    # austin --  train: 43041 test: 6325 \n    # miami -- train: 55029 test:7971\n    # pittsburgh -- train: 43544 test: 6361\n    # dearborn -- train: 24465 test: 3671\n    # washington-dc -- train: 25744 test: 3829\n    # palo-alto -- train:  11993 test:1686\n\n    trajectories sampled at 10HZ rate, input 5 seconds, output 6 seconds\n    \n'

In [24]:
"""
# clean memory
import gc

gc.collect()

torch.cuda.empty_cache()

"""

'\n# clean memory\nimport gc\n\ngc.collect()\n\ntorch.cuda.empty_cache()\n\n'

In [25]:
print(torch.cuda.get_device_name())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GeForce RTX 2060 SUPER


In [26]:
from glob import glob
import pickle
import numpy as np

### Change to requried path to access data locally, too big too push all data into github
#ROOT_PATH = "C:/Users/Administrator/cse151b-spring2022/argo2/"
ROOT_PATH = "D:/School/cse151B/argo2/"

cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc", "palo-alto"]
splits = ["train", "test"]

def get_city_trajectories(city="palo-alto", split="train", normalized=False):
    f_in = ROOT_PATH + split + "/" + city + "_inputs"
    inputs = pickle.load(open(f_in, "rb"))
    inputs = np.asarray(inputs)
    
    outputs = None
    
    if split=="train":
        f_out = ROOT_PATH + split + "/" + city + "_outputs"
        outputs = pickle.load(open(f_out, "rb"))
        outputs = np.asarray(outputs)

        return torch.from_numpy(inputs).float(), torch.from_numpy(outputs).long()

    if split=="test":
    
        return torch.from_numpy(inputs).float(), torch.from_numpy(np.array([]))

    

class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, city: str, split:str, transform=None, device='cpu'):
        super(ArgoverseDataset, self).__init__()
        self.transform = transform
        self.split = split
        self.inputs, self.outputs = get_city_trajectories(city=city, split=split, normalized=False)
        self.inputs = self.inputs.to(device)
        self.outputs = self.outputs.to(device)
    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):

        if self.split == 'train':
            data = (self.inputs[idx], self.outputs[idx])
        if self.split == 'test':
            data = self.inputs[idx]
            
        if self.transform:
            data = self.transform(data)

        return data

# intialize a dataset
city = 'palo-alto' 
split = 'train'
#train_dataset  = ArgoverseDataset(city = city, split = split, device=device)

### Pre Processing

In [27]:
# wrapper function to select proportion of a cities random examples w/o replacement from each city and put all data into one list
# purpose: whole dataset is too big and might be redundant
def randomCitySampler(prop):
    cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc","palo-alto"]

    samples = []
    for c in cities:
        # get city data
        temp_dataset = ArgoverseDataset(city = c, split = "train", device=device)

        numProp = int(len(temp_dataset) * prop)

        # get N number of random indicies
        ind = random.sample(range(0, len(temp_dataset)), numProp)
        #print(ind)
        # push all data indicies into samples list
        for i in ind: 
            samples.append(temp_dataset[i])
    return samples

# generate sequences of length seqLength and specific step size
def sequenceGenerator(data, seqLen=40, stepSize=5):
    newData = []
    for d in data:
        # concat X and Y together
        temp = torch.cat([d[0],d[1]])
        # make X of length SeqLen and Y is next x,y coordinate pair
        for i in range(0,len(temp)-seqLen, stepSize):
            x = temp[i:i + seqLen]
            #flatX = Variable(torch.tensor([item for sublist in x for item in sublist])).to(device)
            flatX = torch.flatten(Variable(torch.tensor(x)).to(device))
            y = temp[i+seqLen]
            newData.append((flatX,y))
            
        
    return newData

In [28]:
### constants for generating Dataset
proportionOfEntireData = 0.001
seqLen = 40
stepSize = 3
batch_sz = 128  # batch size 

In [30]:
# create train dataset, with proportion to actual amount data
sampleTest = randomCitySampler(proportionOfEntireData)
print("length of train dataset:" + str(len(sampleTest)))

# generate sequences
train_seq_data = sequenceGenerator(sampleTest,seqLen, stepSize)
print("# of sequences generated:" + str(len(train_seq_data)))

length of train dataset:201


<ipython-input-27-aa2209eb04b5>:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  flatX = torch.flatten(Variable(torch.tensor(x)).to(device))


# of sequences generated:4824


In [31]:
# shape check
# should be a vector of size (seqlen*2,2) for each example
train_seq_data[0], len(train_seq_data[0][0])

((tensor([-1350.7067, -2750.8108, -1350.3785, -2750.9346, -1349.9807, -2751.0835,
          -1349.5171, -2751.2563, -1348.9944, -2751.4519, -1348.4214, -2751.6663,
          -1347.8091, -2751.8945, -1347.1722, -2752.1311, -1346.5271, -2752.3699,
          -1345.8975, -2752.6016, -1345.3042, -2752.8184, -1344.7107, -2753.0349,
          -1344.1135, -2753.2498, -1343.5172, -2753.4646, -1342.9221, -2753.6792,
          -1342.3304, -2753.8921, -1341.7388, -2754.1033, -1341.1438, -2754.3135,
          -1340.5536, -2754.5222, -1339.9639, -2754.7302, -1339.3729, -2754.9380,
          -1338.7805, -2755.1492, -1338.1881, -2755.3596, -1337.5938, -2755.5688,
          -1336.9995, -2755.7773, -1336.4041, -2755.9829, -1335.8080, -2756.1851,
          -1335.2140, -2756.3811, -1334.6217, -2756.5723, -1334.0299, -2756.7573,
          -1333.4473, -2756.9358, -1332.8748, -2757.1094, -1332.3043, -2757.2795,
          -1331.7285, -2757.4438, -1331.1555, -2757.6030, -1330.5864, -2757.7568,
          -1330.

In [32]:
# create loader
train_loader = DataLoader(train_seq_data,batch_size=batch_sz)

In [33]:
# check shape is correct
train_features, train_labels = next(iter(train_loader))
len(train_features[0]), train_labels[0]
# shape is correct

(80, tensor([-1327.7303, -2758.4739], device='cuda:0'))

### Seq2Seq

In [34]:
# model parameters
learning_rate = 0.001


input_size_encoder = input_size = seqLen*2 #number of features
input_size_decoder = input_size = seqLen*2 #number of features
encoder_embedding_size = 300
decoder_embedding_size = 300
hidden_size = 256 #number of features in hidden state
num_layers = 2 #number of stacked lstm layers

output_size = 2 #number of output classes 
enc_dropout = 0.5
dec_dropout = 0.5

In [35]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
        super(Encoder, self).__init__()
        self.dropout = nn.Dropout(p)
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=p)

    def forward(self, x):
        # x shape: (seq_length, N) where N is batch size

        embedding = self.dropout(self.embedding(x))
        # embedding shape: (seq_length, N, embedding_size)

        outputs, (hidden, cell) = self.rnn(embedding)
        # outputs shape: (seq_length, N, hidden_size)

        return hidden, cell

In [36]:
class Decoder(nn.Module):
    def __init__(
        self, input_size, embedding_size, hidden_size, output_size, num_layers, p
    ):
        super(Decoder, self).__init__()
        self.dropout = nn.Dropout(p)
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=p)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden, cell):
        # x shape: (N) where N is for batch size, we want it to be (1, N), seq_length
        # is 1 here because we are sending in a single word and not a sentence
        x = x.unsqueeze(0)

        embedding = self.dropout(self.embedding(x))
        # embedding shape: (1, N, embedding_size)

        outputs, (hidden, cell) = self.rnn(embedding, (hidden, cell))
        # outputs shape: (1, N, hidden_size)

        predictions = self.fc(outputs)

        # predictions shape: (1, N, length_target_vocabulary) to send it to
        # loss function we want it to be (N, length_target_vocabulary) so we're
        # just gonna remove the first dim
        predictions = predictions.squeeze(0)

        return predictions, hidden, cell

In [37]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_force_ratio=0.5):
        batch_size = source.shape[1]
        target_len = target.shape[0]
        target_vocab_size = 2

        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

        hidden, cell = self.encoder(source)

        # Grab the first input to the Decoder which will be <SOS> token
        x = target[0]

        for t in range(1, target_len):
            # Use previous hidden, cell as context from encoder at start
            output, hidden, cell = self.decoder(x, hidden, cell)

            # Store next output prediction
            outputs[t] = output

            # Get the best word the Decoder predicted (index in the vocabulary)
            #best_guess = output.argmax(1)

            # With probability of teacher_force_ratio we take the actual next word
            # otherwise we take the word that the Decoder predicted it to be.
            # Teacher Forcing is used so that the model gets used to seeing
            # similar inputs at training and testing time, if teacher forcing is 1
            # then inputs at test time might be completely different than what the
            # network is used to. This was a long comment.
            #x = target[t] if random.random() < teacher_force_ratio else best_guess

        return outputs

In [38]:
encoder_net = Encoder(
    input_size_encoder, encoder_embedding_size, hidden_size, num_layers, enc_dropout
).to(device)


In [39]:
decoder_net = Decoder(
    input_size_decoder,
    decoder_embedding_size,
    hidden_size,
    output_size,
    num_layers,
    dec_dropout,
).to(device)

In [40]:
model = Seq2Seq(encoder_net, decoder_net).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

### Training

In [41]:
num_epochs = 10

In [46]:
for epoch in range(num_epochs):
    print(f"[Epoch {epoch} / {num_epochs}]")


    for batch_idx, batch in enumerate(train_seq_data):
        # Get input and targets and get to cuda
        print(batch[1])
        inp_data = batch[0].to(device)
        target = batch[1].to(device)

        # Forward prop
        output = model(inp_data, target)

        # Output is of shape (trg_len, batch_size, output_dim) but Cross Entropy Loss
        # doesn't take input in that form. For example if we have MNIST we want to have
        # output to be: (N, 10) and targets just (N). Here we can view it in a similar
        # way that we have output_words * batch_size that we want to send in into
        # our cost function, so we need to do some reshapin. While we're at it
        # Let's also remove the start token while we're at it
        #output = output[1:].reshape(-1, output.shape[2])
        #target = target[1:].reshape(-1)

        optimizer.zero_grad()
        loss = criterion(output, target)

        # Back prop
        loss.backward()

        # Clip to avoid exploding gradient issues, makes sure grads are
        # within a healthy range
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        # Gradient descent step
        optimizer.step()

        # Plot to tensorboard
        #writer.add_scalar("Training loss", loss, global_step=step)
        #step += 1
        print("Epoch: {0} Loss {1}".format(epoch,loss.item()))


[Epoch 0 / 150]
tensor([-1327.7303, -2758.4739], device='cuda:0')


IndexError: tuple index out of range

### Predicting

In [ ]:
# need to cycle for each test set and use LSTM to predict 
# add results for each city to dataframe 
# concatinate all dataframes
# returns array of DF for each city formatted for kaggle submission
def validation(model):
    cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc","palo-alto"]

    # all the data frames
    allDF = []
    with torch.no_grad():
        for c in cities:
            
            test_dataset = ArgoverseDataset(city=c, split='test', device=device)
            test_loader = DataLoader(test_dataset,batch_size=128)

            cityPredictions = []
            for t in test_loader.dataset:
                model.eval()
                flat = torch.flatten(t)
                currentPred = []

                for i in range(60):
                    #print(flat)
                    #print(len(flat)-seqLen*2)

                    pred = torch.flatten(model(flat[len(flat)-seqLen*2:].view(1,seqLen*2)))

                    #print(flat[len(flat)-seqLen*2:].view(1,80))
                    #print(pred)

                    currentPred.append(pred)

                    #print(torch.flatten(pred).shape)
                    #print(flat.shape)
                    
                    flat = torch.cat((flat,pred),0)
                    #print(flat)

                #print(len(flat[100:]))
                cityPredictions.append(flat[100:].detach().to('cpu').numpy())
        
            df = pd.DataFrame(cityPredictions)
            df.columns = ['v' + str(i) for i in (range(120))]
            df['ID'] = [str(i) + '_' + c for i in (range(len(test_loader.dataset)))]
            allDF.append(df)
            
    return allDF


In [ ]:

tempValDF = validation("model")
